In [1]:
import json
import onnx
import torch
import numpy as np
import pandas as pd
from torch import nn

In [2]:
class ExampleModel(nn.Module):

    def __init__(self):
        super(ExampleModel, self).__init__()

    def forward(self, u, U, X):
        U1 = torch.concat((U[1:, :], u))
        x = torch.stack([U1[-3, 0], U1[-2, 1], U1[-1, 2]]).unsqueeze(0)
        X1 = torch.concat((X[1:, :], x))
        return x, X1, U1

In [3]:
FEATURES = 5
TARGETS = 3
T = 10
# Create three tensors
u = torch.ones((1, FEATURES))
U = torch.ones((T - 1, FEATURES)) * torch.arange(T, 1, -1).unsqueeze(1)
X = torch.ones(T, TARGETS)
print("Input shapes", u.shape, U.shape, X.shape)

# Create the model
model = ExampleModel()

# Run the model
output = model(u, U, X)
print("Output shapes", *[i.shape for i in output])

Input shapes torch.Size([1, 5]) torch.Size([9, 5]) torch.Size([10, 3])
Output shapes torch.Size([1, 3]) torch.Size([10, 3]) torch.Size([9, 5])


In [4]:
model_name = "example4"
onnx_model_path = f"{model_name}.onnx"
# Set the model to evaluation mode
model.eval()

# Save the model in ONNX format
torch.onnx.export(
    model,
    (u, U, X),
    f"{model_name}.onnx",
    verbose=False,
    input_names=["u", "U", "X"],
    output_names=["x", "X1", "U1"],
)

# Load the model
onnx_model = onnx.load(onnx_model_path)

# Check the model
onnx.checker.check_model(onnx_model)

# Add description to the model
onnx_model.graph.doc_string = "Example to test FMU with local variables."

# Add metadata to the model
onnx_model.producer_name = "ExampleModel"
onnx_model.producer_version = "0.0.1"
onnx_model.domain = "example"
onnx_model.model_version = 1

# Save the model
onnx.save(onnx_model, onnx_model_path)


## Generating model description

Create and save the model description to be provided to ONNX2FMU. If start
values are not provided, ONNX2FMU sets them to 1.0. For the sake of this example,
we want to set start values to 0.0, otherwise, the values returned by the FMU
during the first two iterations differ from the values of the pure-Python model.

In [5]:
model_description = {
    "name": "example4",
    "description": "Example to test FMU with local variables.",
    "FMIVersion": "2.0",
    "inputs": [
        {
            "name": "u",
            "description": "A vector of control variables at time t.",
            "start": [
                0.0,
                0.0,
                0.0,
                0.0,
                0.0
            ]
        },
    ],
    "outputs": [
        {
            "name": "x",
            "description": "The state of the system at time t+1."
        }
    ],
    "locals": [
        {
            "nameIn": "X",
            "nameOut": "X1",
            "description": "The history of states from t-N to t."
        },
        {
            "nameIn": "U",
            "nameOut": "U1",
            "description": "The history of control variables frmo t-N to t-1.",
            "start": [
                0.0,
                0.0,
                0.0,
                0.0,
                0.0
            ]
        }
    ]
}

# Save model description for FMI 2.0
with open(f"{model_name}Description.json", "w", encoding="utf-8") as f:
    json.dump(model_description, f, indent=4)

model_description["FMIVersion"] = "3.0"

# Save model description for FMI 3.0
with open(f"{model_name}DescriptionFMI3.json", "w", encoding="utf-8") as f:
    json.dump(model_description, f, indent=4)


## Generating input file and output for testing

In [6]:
time_steps = 100
U_hist = np.ones((time_steps, FEATURES)) * np.arange(time_steps)[:, None]
columns = [f"u_0_{i}" for i in range(FEATURES)]
index = pd.Index(data=np.arange(time_steps), name='time')
df = pd.DataFrame(
    data=U_hist,
    columns=columns,
    index=index
)
df.to_csv(f"Example4_in.csv")

results = torch.empty((time_steps, TARGETS))
U = torch.zeros((T - 1, FEATURES))
X = torch.zeros((T, TARGETS))
for i, u in enumerate(U_hist):
    x, X1, U1 = model(
        torch.tensor(u).unsqueeze(0), U, X
    )
    U, X = U1, X1
    results[i] = x

output = pd.DataFrame(
    data=results.detach().numpy(),
    columns=[f"x_0_{i}" for i in range(results.shape[1])],
    index=index
)

output.to_csv("Example4_ref.csv")

## Generate the FMU

In [7]:
from onnx2fmu.app import build
# FMI 2.0
build(
    model_path=onnx_model_path,
    model_description_path=f"{model_name}Description.json",
    target_folder="temp",
    destination="example4FMI2.fmu"
)

# FMI 3.0
build(
    model_path=onnx_model_path,
    model_description_path=f"{model_name}DescriptionFMI3.json",
    target_folder="temp",
    destination="example4FMI3.fmu"
)

2025-11-24 10:29:41.936 | INFO     | onnx2fmu.app:compile:246 - Call cmake -S temp -B temp/build -D MODEL_NAME=example4 -D FMI_VERSION=2


-- The C compiler identification is GNU 13.3.0
-- The CXX compiler identification is GNU 13.3.0
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Check for working C compiler: /usr/bin/cc - skipped
-- Detecting C compile features
-- Detecting C compile features - done
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /usr/bin/c++ - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- ONNX Runtime archive already exists at /tmp/onnxruntime/onnxruntime-linux-x64-1.20.1.tgz


2025-11-24 10:29:43.210 | INFO     | onnx2fmu.app:compile:248 - CMake build cmake --build temp/build --config Release


-- ONNX Runtime extraction successful.
-- Configuring done (1.1s)
-- Generating done (0.0s)
-- Build files have been written to: /home/michele/onnx2fmu/examples/example4/temp/build
[ 20%] Building C object CMakeFiles/example4.dir/example4/model.c.o
[ 40%] Building C object CMakeFiles/example4.dir/src/fmi2Functions.c.o
[ 60%] Building C object CMakeFiles/example4.dir/src/cosimulation.c.o
[ 80%] Building C object CMakeFiles/example4.dir/src/ortUtils.c.o
[100%] Linking C shared library temp/example4/binaries/linux64/example4.so


2025-11-24 10:29:45.640 | INFO     | onnx2fmu.app:compile:246 - Call cmake -S temp -B temp/build -D MODEL_NAME=example4 -D FMI_VERSION=3


modelDescription.xml
binaries
binaries/linux64
binaries/linux64/libonnxruntime.so.1.20.1
binaries/linux64/example4.so
binaries/linux64/libonnxruntime.so.1
binaries/linux64/libonnxruntime_providers_shared.so
binaries/linux64/libonnxruntime.so
sources
sources/cosimulation.h
sources/all.c
sources/model.c
sources/cosimulation.c
sources/config.h
sources/onnxruntime_c_api.h
sources/model.h
sources/ortUtils.h
sources/fmi2Functions.c
resources
resources/model.onnx
[100%] Built target example4
-- The C compiler identification is GNU 13.3.0
-- The CXX compiler identification is GNU 13.3.0
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Check for working C compiler: /usr/bin/cc - skipped
-- Detecting C compile features
-- Detecting C compile features - done
-- Detecting CXX compiler ABI info


2025-11-24 10:29:46.615 | INFO     | onnx2fmu.app:compile:248 - CMake build cmake --build temp/build --config Release


-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /usr/bin/c++ - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- ONNX Runtime archive already exists at /tmp/onnxruntime/onnxruntime-linux-x64-1.20.1.tgz
-- ONNX Runtime extraction successful.
-- Configuring done (0.9s)
-- Generating done (0.0s)
-- Build files have been written to: /home/michele/onnx2fmu/examples/example4/temp/build
[ 20%] Building C object CMakeFiles/example4.dir/example4/model.c.o
[ 40%] Building C object CMakeFiles/example4.dir/src/fmi3Functions.c.o
[ 60%] Building C object CMakeFiles/example4.dir/src/cosimulation.c.o
[ 80%] Building C object CMakeFiles/example4.dir/src/ortUtils.c.o
[100%] Linking C shared library temp/example4/binaries/x86_64-linux/example4.so
modelDescription.xml
binaries
binaries/x86_64-linux
binaries/x86_64-linux/libonnxruntime.so.1.20.1
binaries/x86_64-linux/example4.so
binaries/x86_64-linux/libonnxruntime.so.1
binaries/x86_64-linux/

## Test the FMU using FMPy

In [8]:
from fmpy import simulate_fmu

input = np.genfromtxt(f"{model_name.capitalize()}_in.csv", delimiter=",", names=True)

# Test FMI 2.0
resultsFMI2 = simulate_fmu(
    f"{model_name}FMI2.fmu",
    start_time=0,
    stop_time=100,
    input=input,
    validate=True,
    output_interval=1
)

In [ ]:
# Time is a column in the results object, and it set to index
df_out = pd.DataFrame(
    data=resultsFMI2,
).set_index('time')

In [12]:
# Compare df_out and output data frames to check that they are equal
pd.testing.assert_frame_equal(
    df_out.loc[1:].set_index(pd.Index(range(0, len(output)), name='time')),
    output,
    check_dtype=False
)